In [ ]:
import numpy as np
import pandas as pd
import spacy
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from spacy.lang.pt.examples import sentences

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:
np.random.seed(500)
nlp = spacy.load('pt_core_news_sm')

In [ ]:
#Seleciona a linguagem do corretor para português e seleciona o dicionário personalizado.
import re
#spell.word_frequency.load_text_file('palavras.txt')

def removePontuacao(sentenca): 
    sentenca = re.sub(u'[^a-zA-ZáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', r' ',sentenca)
    sentenca = re.sub(r'[?|!|\'|"|#]',r'',sentenca)
    sentenca = re.sub(r'[.|,|)|(|\|/]',r' ',sentenca)
    sentenca = sentenca.replace("\n"," ")
    return sentenca

def removeruidos(sentenca):
    #sentenca = sentenca.replace("username","")
    sentenca = sentenca.replace("hashtag","")
    sentenca = sentenca.replace("retweeet","")
    return sentenca

def corrige_internetes(sentenca):
    norm = normaliser.Normaliser(sanitize = True, capitalize_inis=True, capitalize_pns=True, capitalize_acs=True)
    norm_sentence = norm.normalise(sentenca)
    return norm_sentence

def deEmojify(sentenca):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',sentenca)

In [ ]:
Corpus = pd.read_csv(r"tweets_eleicao_enelvo_raw.csv",encoding='utf-8')

In [ ]:
Corpus["text"] = Corpus["text"].apply(lambda x: removeruidos(x))
Corpus["text"] = Corpus["text"].apply(lambda x: deEmojify(x))

In [ ]:
Corpus.columns = ['index', 'user', 'Tweet', 'Polaridade', 'Localizacao']

In [ ]:
#PRÉ-PROCESSAMENTO
# Remover linhas vazias.
#Corpus['Tweet'].dropna(inplace=True)

In [ ]:
# Padroniza todo o texto em caixa baixa (letras minúsculas)
#Corpus['Tweet'] = [entry.lower() for entry in Corpus['Tweet']]

In [ ]:
# Remove @RT de retweets
#Corpus['Tweet'] = [re.sub(r'^RT[\s]+', '', entry) for entry in Corpus['Tweet']]

In [ ]:
# Remove hiperlinks
#Corpus['Tweet'] = [re.sub(r'https?:\/\/.*[\r\n]*', '', entry) for entry in Corpus['Tweet']]

In [ ]:
Corpus

In [ ]:
corpus_bert = Corpus.copy()

In [ ]:
# Tokenização: Cada tweet é dividido em um array de palavras
Corpus['Tweet']= [word_tokenize(entry) for entry in Corpus['Tweet']]

In [ ]:
Corpus

In [ ]:
# Remover stop-words e aplicar stemming
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['Tweet']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('portuguese') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Final_words = [token.lemma_ for token in nlp(str(Final_words)) if token.pos_ == 'VERB']
    Corpus.loc[index,'text_final'] = str(Final_words)
    #print([token.lemma_ for token in nlp(str(Final_words)) if token.pos_ == 'VERB'])
#print(token.lemma_ for token in Corpus['Tweet'] if token.pos_ == 'VERB')
#print(Corpus['text_final'])

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['Polaridade'],test_size=0.1, random_state=2020)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
#Padrões TF-IDF
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
#print(Tfidf_vect.vocabulary_)

#print(Train_X_Tfidf)

# ajustar o conjunto de dados de treinamento no classificador NB
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# prever os rótulos no conjunto de dados de validação
predictions_NB = Naive.predict(Test_X_Tfidf)

In [ ]:
#imprimir na tela a acurácia, f-measure, precisão e recall
print("Naive Bayes Accuracy: ",accuracy_score(predictions_NB, Test_Y)*100)
print("Naive Bayes F-Measure: ",f1_score(predictions_NB, Test_Y, average="macro")*100)
print("Naive Bayes Precision: ",precision_score(predictions_NB, Test_Y, average="macro")*100)
print("Naive Bayes Recall: ",recall_score(predictions_NB, Test_Y, average="macro")*100)

In [ ]:
# Classificador SVM
# ajustar o conjunto de dados de treinamento no classificador SVM
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# prever os rótulos no conjunto de dados de validação
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [ ]:
#imprimir na tela a acurácia, f-measure, precisão e recall
print("SVM Accuracy: ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM F-Measure: ",f1_score(predictions_SVM, Test_Y, average="macro")*100)
print("SVM Precision: ",precision_score(predictions_SVM, Test_Y, average="macro")*100)
print("SVM Recall: ",recall_score(predictions_SVM, Test_Y, average="macro")*100)

In [ ]:
corpus_bert.columns = ['user', 'text', 'sentiment', 'location']

In [ ]:
corpus_bert

In [ ]:
corpus_bert["sentiment"] = corpus_bert["sentiment"].replace("Positivo", "Positiva")
corpus_bert["sentiment"] = corpus_bert["sentiment"].replace("Neutro", "Neutra")
corpus_bert["sentiment"] = corpus_bert["sentiment"].replace("Negativo", "Negativa")